<a href="https://colab.research.google.com/github/wesslen/rag-workflow/blob/main/01_initial_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!uv pip install --system llama-index llama-index-embeddings-huggingface llama-index-llms-openai-like

In [2]:
import os
from google.colab import userdata

API_KEY = userdata.get('API_KEY')
BASE_URL = userdata.get('BASE_URL')

In [3]:
# take WF Code of Conduct
!wget -O text.txt https://r.jina.ai/https://www.wellsfargo.com/assets/pdf/about/corporate/code-of-conduct.pdf

--2025-01-11 23:32:34--  https://r.jina.ai/https://www.wellsfargo.com/assets/pdf/about/corporate/code-of-conduct.pdf
Resolving r.jina.ai (r.jina.ai)... 172.67.70.54, 104.26.10.242, 104.26.11.242, ...
Connecting to r.jina.ai (r.jina.ai)|172.67.70.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41541 (41K) [text/plain]
Saving to: ‘text.txt’

text.txt            100%[===================>]  40.57K  --.-KB/s    in 0.004s  

2025-01-11 23:32:35 (11.1 MB/s) - ‘text.txt’ saved [41541/41541]



In [4]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model="/models/NousResearch/Meta-Llama-3.1-8B-Instruct", api_base=BASE_URL, api_key=API_KEY)
# response = llm.complete("Hello World!")
# print(str(response))

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

# Loading text files in root
documents = SimpleDirectoryReader(".").load_data()

Settings.chunk_size = 100
Settings.chunk_overlap = 20

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [7]:
from llama_index.core import Settings

# To turn off generation
# Settings.llm = None

rag_application = index.as_query_engine(similarity_top_k=5, llm=llm)

In [8]:
# An example input to your RAG application
user_input = "What are Wells Fargo's policies on lobbying and pay-to-play laws?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
response_object = rag_application.query(user_input)

# Process the response object to get the output string
# and retrieved nodes
if response_object is not None:
    actual_output = response_object.response
    retrieval_context = [node.get_content() for node in response_object.source_nodes]

In [9]:
actual_output

' Wells Fargo does not use company funds for campaign contributions, candidate campaign committees, political'

In [10]:
retrieval_context

['> 13 of 19 Code of Conduct\n> TOC  Decision making  Speak Up Introduction  Do what is right  Closing\n\nAnti-bribery and anti-corruption \n\nWells Fargo does not tolerate bribery or corruption in any form. Such conduct is against the law and the company prohibits it. Further, employees are prohibited from offering, accepting, or utilizing third parties to facilitate bribes.',
 'Financial crimes and money laundering \n\nAs a global financial institution, Wells Fargo has important responsibilities to help combat money laundering and other financial crimes including tax evasion, terrorist financing, identity theft, bribery, corruption, sanctions evasion, and fraud. We are committed to complying with all applicable global laws and regulations related to anti-money laundering, sanctions, countering the financing of terrorism, bribery, and corruption, and following applicable tax rules.',
 '> 8 of 19 Code of Conduct\n> TOC  Decision making  Speak Up Introduction  Do what is right  Closing\